# Accessing IMERG-Late Data in the Cloud (Algeria)

Parts of northern Algeria and Tunisia experienced flash floods in May 2023. Let's use the IMERG-Late product to quantify the total precipitation that fell across the country on one day in that region.

In [ ]:
import earthaccess
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

## Using `earthaccess`

While IMERG-Final is produced in 30-minute intervals, today we'll be using a version of the data that have been aggregated to daily time steps.

In [ ]:
results = earthaccess.search_data(
    short_name = 'GPM_3IMERGDL',
    temporal = ('2023-05-25', '2023-05-25'))

results[0]

In [ ]:
# NOTE: open() requires a sequence of file references
files = earthaccess.open(results)
files

In [ ]:
ds = xr.open_dataset(files[0])
ds

There are a lot of different variables in this dataset, all described in [the IMERG-Late documentation.](https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGHHL_06/summary). We'll use `precipitationCal`, which is the gauge-calibrated, multi-satellite estimate that NASA recommends for general use.

In [ ]:
ds['precipitationCal']

In [ ]:
ds['precipitationCal'].plot()

That looks weird! Any ideas about what is wrong?

We need to rotate the plot so that the rows of the image correspond to latitude bands. **Specifically, we need to tell `xarray` that longitude ("lon") should span the X-axis and latitude ("lat") should span the Y-axis.**

In [ ]:
ds['precipitationCal'].plot(x = 'lon', y = 'lat')

This looks better. IMERG doesn't make estimates over ice and snow-covered area, so you may recognize parts of Iceland, Norway, and Sweden in the upper-center of the image.

Why is most of the image dark?

The easiest way to see more detail in the image is to tell `xarray` to stretch the colorbar so that extreme values don't dominate; we do this with `robust = True`.

In [ ]:
ds['precipitationCal'].plot(x = 'lon', y = 'lat', robust = True)

Alternatively, we could tell `xarray` what the maximum value assigned to a color should be, using the `vmax` keyword argument. There's a corresponding `vmin` argument for the minimum value.

In [ ]:
ds['precipitationCal'].plot(x = 'lon', y = 'lat', vmax = 15)

## Zooming In

How can we use these data for local applications? We need to figure out a way to focus the map on a smaller area.

Python's built-in `slice()` function can be used with the `sel()` method of an `xarray` DataArray in order to slice a larger array into a smaller array. Here, we focus on a small, rectangular bounding box that inclues Algiers.

In [ ]:
precip = ds['precipitationCal'].sel(lat = slice(30, 40), lon = slice(-6, 6))

precip.plot(x = 'lon', y = 'lat')

## Using `cartopy`

It's a good idea to verify that we're mapping the right part of the world, especially since the IMERG data are rotated. `cartopy` is a Python library that provides some additional mapping tools.

Below, I changed the colormap, `cmap`, so that it is easier to see the dark coastlines on top of the precipitation data.

Note that Plate-Carree is just a fancy term for an equirectangular coordinate system, i.e., a longitude-latitude plot.

In [ ]:
from cartopy import crs

proj = crs.PlateCarree()
style = {
    'projection': proj,
    'facecolor': 'lightgray',
}

plot = precip.plot(subplot_kws = style, transform = proj, cmap = 'magma_r', x = 'lon', y = 'lat')
plot.axes.coastlines()

So, how much rain fell around Algiers on this day?

In [ ]:
precip.sel(lon = 3.059, lat = 36.754, method = 'nearest').values

That's a good amount of rain for this region in a single day, but from the map above it's clear that there are areas near Algiers that received more rain. What's the maximum rainfall total for the coast of Algiers?

In [ ]:
precip.sel(lon = slice(3, 3.2), lat = slice(36.5, 36.8)).max()

Just a reminder, because the `values` are returned as a NumPy array, we can do math on these arrays, treating them as if they were just numbers. So, a conversion from mm to meters is easy:

In [ ]:
precip.sel(lon = slice(3, 3.2), lat = slice(36.5, 36.8)).max() / 1000